In [1]:
!pip install -qr requirements.txt


[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# common
import os
import boto3
import botocore

# local
from integrated_gradients import *
from visualization import *

# third party 
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer
)

In [10]:
torch.__version__

'2.0.1+cu118'

In [2]:
tokenizer = AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")
model = AutoModelForSequenceClassification.from_pretrained("lvwerra/distilbert-imdb")

In [3]:
model_inputs = tokenizer(text = """
                i went to see this movie with a bunch of friends one night.
                I didn't really hear much about it. So I wasn't expecting anything.
                But after I saw it, I really liked it. Nicolas Cage and the rest of
                the cast were very good. But I do have to say Giovanni Ribisi's acting
                performace did need a little perking up. But such a small flaw, it
                could be overrided. <br /><br />Gone In 60 Seconds is about a retired
                car thief who must boost 60 rare and exotic cars in one night to save
                his brother's life. The movie is in no way predictable. So the ending
                should be a suprise. Think it's just another, fast car driving movie?
                Well you are partially right. There is much more to it. Everyone should
                take a look at this movie.
               """, 
                return_tensors="pt")

In [13]:
ig_enabled_model = IGWrapper(model).eval()

scores = ig_enabled_model(model_inputs['input_ids'],
                          model_inputs['attention_mask'],
                          baseline=torch.zeros([1, 181, 768]),
                          )

In [5]:
def get_dummy_baseline(model, input_ids):
    return torch.zeros_like(model.base_model.embeddings.word_embeddings(model_inputs['input_ids']))

In [6]:
dummy_baseline = get_dummy_baseline(model, model_inputs['input_ids'])

In [14]:
traced_ig_enabled_model = torch.jit.trace(
        ig_enabled_model,
        (model_inputs['input_ids'],
        model_inputs['attention_mask'],
        dummy_baseline)
    )
traced_ig_enabled_model.save('models/model/1/model.pt')

step: 0/5
step: 1/5
step: 2/5
step: 3/5
step: 4/5
step: 5/5
step: 0/5
step: 1/5
step: 2/5
step: 3/5
step: 4/5
step: 5/5
step: 0/5
step: 1/5
step: 2/5
step: 3/5
step: 4/5
step: 5/5


In [9]:
traced_ig_enabled_model = torch.jit.load('models/model/1/model.pt')

scores = traced_ig_enabled_model(
    model_inputs['input_ids'],
    model_inputs['attention_mask'],
    baseline=torch.zeros([1, 181, 768])   
)

## Save model to MinIO bucket

In [16]:
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)

def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [17]:
upload_directory_to_s3("models", "models")

models/model/config.pbtxt -> models/model/config.pbtxt
models/model/1/model.pt -> models/model/1/model.pt
models/model/.ipynb_checkpoints/config-checkpoint.pbtxt -> models/model/.ipynb_checkpoints/config-checkpoint.pbtxt


In [14]:
list_objects("models")

models/model/.ipynb_checkpoints/config-checkpoint.pbtxt
models/model/1/model.pt
models/model/config.pbtxt
